In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_folder = '/content/drive/MyDrive/datasets/dataset/rose'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
root_dir = '/content/drive/MyDrive/datasets/dataset/preprocessed'
os.makedirs(root_dir, exist_ok=True)

In [ ]:
classes = ['Fresh Leaf', 'Downy mildew', 'Black Spot']

In [ ]:
train_dir = os.path.join(root_dir, 'train')
test_dir = os.path.join(root_dir, 'test')
val_dir = os.path.join(root_dir, 'validation')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

In [ ]:
for class_name in classes:
    class_dir = os.path.join(dataset_folder, class_name)
    images = os.listdir(class_dir)

    # Split the images into train, validation, and test sets
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)
    train_images, val_images = train_test_split(train_images, test_size=0.2, random_state=42)

    # Create subdirectories for each class in the train, validation, and test sets
    train_class_dir = os.path.join(train_dir, class_name)
    val_class_dir = os.path.join(val_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(val_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)

    # Move the images to the corresponding train, validation, and test directories
    for train_image in train_images:
        src_path = os.path.join(class_dir, train_image)
        dst_path = os.path.join(train_class_dir, train_image)
        shutil.copy(src_path, dst_path)

    for val_image in val_images:
        src_path = os.path.join(class_dir, val_image)
        dst_path = os.path.join(val_class_dir, val_image)
        shutil.copy(src_path, dst_path)

    for test_image in test_images:
        src_path = os.path.join(class_dir, test_image)
        dst_path = os.path.join(test_class_dir, test_image)
        shutil.copy(src_path, dst_path)

print("Dataset split and directories created successfully!")

In [ ]:
batch_size=32

In [ ]:
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
train_generator = datagen.flow_from_directory(
    dataset_folder,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 4343 images belonging to 3 classes.


In [ ]:
val_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
val_generator = val_datagen.flow_from_directory(
    dataset_folder,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 4343 images belonging to 3 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(
    dataset_folder,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 4343 images belonging to 3 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(
    dataset_folder,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 4343 images belonging to 3 classes.


# CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
num_classes =3
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 128)      0

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
epochs = 10
model.fit(train_generator,
          validation_data=val_generator,
          epochs=epochs)

Epoch 1/10
136/136 [==============================] - 791s 6s/step - loss: 0.6323 - accuracy: 0.6882 - val_loss: 0.4752 - val_accuracy: 0.7610
Epoch 2/10
136/136 [==============================] - 167s 1s/step - loss: 0.4728 - accuracy: 0.7644 - val_loss: 0.3631 - val_accuracy: 0.8407
Epoch 3/10
136/136 [==============================] - 163s 1s/step - loss: 0.4243 - accuracy: 0.8077 - val_loss: 0.4052 - val_accuracy: 0.8158
Epoch 4/10
136/136 [==============================] - 163s 1s/step - loss: 0.3635 - accuracy: 0.8476 - val_loss: 0.3396 - val_accuracy: 0.8508
Epoch 5/10
136/136 [==============================] - 162s 1s/step - loss: 0.3906 - accuracy: 0.8192 - val_loss: 0.2583 - val_accuracy: 0.8952
Epoch 6/10
136/136 [==============================] - 162s 1s/step - loss: 0.3156 - accuracy: 0.8688 - val_loss: 0.2172 - val_accuracy: 0.9074
Epoch 7/10
136/136 [==============================] - 162s 1s/step - loss: 0.3340 - accuracy: 0.8524 - val_loss: 0.3721 - val_accuracy: 0.8218

In [ ]:
model.evaluate(test_generator)

136/136 [==============================] - 60s 438ms/step - loss: 0.2087 - accuracy: 0.9171


[0.20865929126739502, 0.9171079993247986]

In [ ]:
predictions = model.predict(test_generator)

136/136 [==============================] - 59s 436ms/step


**ResNet50**

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [ ]:
model_resnet = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model_resnet.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
epochs = 30
model_resnet.fit(train_generator,
          validation_data=val_generator,
          epochs=epochs)

In [ ]:
model.evaluate(test_generator)

136/136 [==============================] - 65s 480ms/step - loss: 0.1158 - accuracy: 0.9622


[0.11583661288022995, 0.962238073348999]

# inception v3

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score, roc_auc_score

In [ ]:
image_size=(224,224)

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
num_classes = 3

batch_size = 32

epochs = 10

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
HISTORY = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)

Epoch 1/10
135/135 [==============================] - 259s 2s/step - loss: 0.2497 - accuracy: 0.9202 - val_loss: 17.2763 - val_accuracy: 0.4111
Epoch 2/10
135/135 [==============================] - 200s 1s/step - loss: 0.1106 - accuracy: 0.9729 - val_loss: 0.0613 - val_accuracy: 0.9852
Epoch 3/10
135/135 [==============================] - 186s 1s/step - loss: 0.0634 - accuracy: 0.9845 - val_loss: 0.3567 - val_accuracy: 0.8544
Epoch 4/10
135/135 [==============================] - 200s 1s/step - loss: 0.0606 - accuracy: 0.9828 - val_loss: 0.0681 - val_accuracy: 0.9780
Epoch 5/10
135/135 [==============================] - 195s 1s/step - loss: 0.0493 - accuracy: 0.9875 - val_loss: 0.1051 - val_accuracy: 0.9648
Epoch 6/10
135/135 [==============================] - 175s 1s/step - loss: 0.0498 - accuracy: 0.9856 - val_loss: 0.0170 - val_accuracy: 0.9949
Epoch 7/10
135/135 [==============================] - 173s 1s/step - loss: 0.0356 - accuracy: 0.9907 - val_loss: 0.1117 - val_accuracy: 0.979

In [ ]:
loss, accuracy = model.evaluate(test_generator)
print('Test Loss :', loss)
print('Test Accuracy:',accuracy)

NameError: ignored

# **Cnn+lstm**

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(254, 254, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(254, activation="relu"),
    keras.layers.Reshape((1, 254)),  # Add a reshape layer to add the third dimension (timesteps)
    keras.layers.LSTM(128),
    keras.layers.Dense(3, activation="softmax")
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=10)

In [ ]:
model.evaluate(test_generator)